In [1]:
## 1. Import Libraries and local dependencies, such as the NeuralNetwork class

In [12]:
from neural_lib.neural_network import NeuralNetwork

import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
## 2. Load and Preprocess Data

In [13]:
def load_and_preprocess(csv_path):
    df = pl.read_csv(csv_path)

    continuos_features = [
        'Age', 'AnnualIncome', 'NumberOfPurchases',
        'TimeSpentOnWebsite', 'DiscountsAvailed'
    ]
    categorical_features = ['ProductCategory']
    passthrough_feats = ['Gender', 'LoyaltyProgram']

    features = continuos_features + categorical_features + passthrough_feats
    X_raw = df.select(features).to_numpy()
    y = df.select('PurchaseStatus').to_numpy().flatten()

    preprocessor = ColumnTransformer([
        ('scale', StandardScaler(), list(range(0, len(continuos_features)))),
        ('onehot', OneHotEncoder(sparse_output=False), [len(continuos_features)]),
    ], remainder='passthrough')

    X = preprocessor.fit_transform(X_raw)
    return X, y


In [5]:
## 3. Define Model and Helper Functions

In [14]:
def train_model(X_train, y_train, hidden_layers, epochs, activation_function):
    input_size = X_train.shape[1]
    neural_network = NeuralNetwork(
        num_input_neurons=input_size,
        hidden_layer_sizes=hidden_layers,
        activation_function=activation_function,
        num_output_neurons=1,
        output_activation_function='sigmoid',
        loss_function='binary'
    )

    neural_network.train(X_train, y_train, epochs)
    return neural_network

def evaluate_model(neural_network, X_test, y_test):
    predictions = neural_network.predict(X_test).flatten()
    results = (predictions >= 0.5).astype(int)
    return np.mean(results == y_test)

def get_params():
    return [
        {'hidden_layers': [32, 16], 'activation_function': 'relu',    'epochs': 100},
        {'hidden_layers': [32, 16], 'activation_function': 'tanh',    'epochs': 100},

        {'hidden_layers': [64, 32, 16], 'activation_function': 'relu',    'epochs': 200},
        {'hidden_layers': [64, 32, 16], 'activation_function': 'tanh',    'epochs': 200},

        {'hidden_layers': [128, 64, 32], 'activation_function': 'relu',    'epochs': 300},
        {'hidden_layers': [128, 64, 32], 'activation_function': 'tanh',    'epochs': 300},
    ]

In [7]:
## Evaluate Models

In [15]:
X, y = load_and_preprocess('./../data/customer-purchase-behavior.csv')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

results = []
for params in get_params():
    hidden_layers = params['hidden_layers']
    epochs = params['epochs']
    activation_function = params['activation_function']

    neural_network = train_model(
        X_train, y_train,
        hidden_layers,
        epochs,
        activation_function
    )
    accuracy = evaluate_model(neural_network, X_test, y_test)
    print(f'Hidden Layers: {hidden_layers}, Epochs: {epochs}, Activation Function: {activation_function} -> Accuracy: {accuracy:.2f}%')

Hidden Layers: [32, 16], Epochs: 100, Activation Function: relu -> Accuracy: 0.77%
Hidden Layers: [32, 16], Epochs: 100, Activation Function: tanh -> Accuracy: 0.72%
Hidden Layers: [64, 32, 16], Epochs: 200, Activation Function: relu -> Accuracy: 0.70%
Hidden Layers: [64, 32, 16], Epochs: 200, Activation Function: tanh -> Accuracy: 0.68%
Hidden Layers: [128, 64, 32], Epochs: 300, Activation Function: relu -> Accuracy: 0.82%
Hidden Layers: [128, 64, 32], Epochs: 300, Activation Function: tanh -> Accuracy: 0.79%
